In [1]:
import re
import pandas as pd

In [2]:
re_tags = re.compile('<.*?>') # re to remove all the tags
re_patent_number = re.compile('file="([^-]*)-') # re to extract patent number

re_paranthesis = re.compile('\(.*?\)') # removing stuff in paranthesis

In [3]:
number_of_patents = 0
number_of_design_patents = 0
patent_data = {"Patent_Number":[], "Description":[]};
description_flag = 0
description_string = 'x'
with open('ipg230207.xml', 'r') as f:
    for line in f:
        
#         if number_of_patents <= 1:
            #checking the number of patents
        if line == '<?xml version="1.0" encoding="UTF-8"?>\n':
            number_of_patents += 1

        #extract patent number
        if '<us-patent-grant lang="EN"' in line:
            patent_num = re_patent_number.findall(line)
            if 'USD0' in patent_num[0]:
                number_of_design_patents += 1

            patent_data["Patent_Number"].append(patent_num[0])

        if '<description id="description">' in line:
            description_string = ''
            description_flag = 1

        if line == '</description>\n':
            description_flag = 0
            patent_data["Description"].append(description_string)

        else:
            if description_flag == 1:
                line = re.sub(re_tags, '', line)
                description_string += line
#             elif description_string == '' or description_string == 'x':
#                 patent_data["Description"].append('')
                
                

In [4]:
print(number_of_patents)

6007


In [5]:
len(patent_data["Description"])

5882

In [6]:
for idx, descr in enumerate(patent_data["Description"]):
#     if 'USD0' in patent_data["Patent_Number"][idx]:
#         patent_data["Description"][idx] = None
#     else:
    new_descr = ''
    new_descr_however = ''
    if 'however' in descr or 'However' in descr:
        flag_however = 0
        for line in descr.split('\n'):
            if len(line) > 20 and 'FIG' not in line and '&#x' not in line:# and 'THE' not in line and 'SUMMARY' not in line:
                
                if ('however' in line or 'However' in line) and flag_however == 0:
                    flag_however = 1
                    new_descr = line
                    continue
                if flag_however < 3:
                    new_descr += ' ' + line
                    flag_however += 1
                if flag_however == 3:
                    flag_however = 0
    else:
        for line in descr.split('\n'):
            if len(line) > 20 and 'FIG' not in line and '&#x' not in line: #and 'THE' not in line and 'SUMMARY' not in line:
                new_descr += ' ' + line

    if new_descr == '' or 'USD0' in patent_data["Patent_Number"][idx] or 'USPP0'in patent_data["Patent_Number"][idx]:
        patent_data["Description"][idx] = None
    else:
        new_descr = str(new_descr).strip()
        new_descr = re.sub(re_paranthesis, '', new_descr)
        patent_data["Description"][idx] = new_descr
#     print('lalala')


# skip USD - design patents - done
# evaluation of how well it describes - this week - done
# however, 3 paragraphs - done
# use examples while prompting - example - next week
# get 100 patents summrized - this week - done
# human problem - connection with the summary - last week? - this week

In [7]:
patent_data_df = pd.DataFrame.from_dict(patent_data)

In [8]:
patent_data_df.describe()

,Patent_Number,Description
count,5882,5312
unique,5882,5297
top,USD0977218,CROSS-REFERENCE TO RELATED APPLICATIONS/INCORP...
freq,1,3


In [9]:
patent_data_df = patent_data_df.dropna()
patent_data_df.describe()

,Patent_Number,Description
count,5312,5312
unique,5312,5297
top,USRE049409,CROSS-REFERENCE TO RELATED APPLICATIONS/INCORP...
freq,1,3


In [15]:
count_20 = 0
count_1500 = 0
patent_no_high_words = []
patent_no_list = []
num_of_words = 0
patent_data['length'] = []
for idx, line in enumerate(patent_data['Description']):
    if line:
        words = line.split(' ')
        patent_data['length'].append(len(words)) 
        if len(words) < 20:
#             print(len(words))
#             print(patent_data['Patent_Number'][idx])
            count_20 += 1
        elif len(words) > 1500:
            count_1500 += 1
            patent_no_high_words.append(patent_data['Patent_Number'][idx])
        else:
            num_of_words += len(words)
            patent_no_list.append(patent_data['Patent_Number'][idx])
        
print('the count is of patent descriptions with less than 20 words = '+ str(count_20))
print('the count is of patent descriptions with more than 1500 words = '+ str(count_1500))
print('total count of tokens from usable patents = '+ str(num_of_words))
print('number of patents that can be used now: '+ str(len(patent_no_list)))

the count is of patent descriptions with less than 20 words = 0
the count is of patent descriptions with more than 1500 words = 4196
total count of tokens from usable patents = 868463
num of patents that can be used now: 1116


In [16]:
patents_list_100 = patent_no_list[:100]
print(len(patents_list_100))
patent_data_100 = patent_data_df[patent_data_df['Patent_Number'].isin(patents_list_100)]

100


In [14]:
patent_data_100.head()

,Patent_Number,Description
570,USRE049409,"In the present embodiment, the display unit 40..."
572,USRE049411,The locking catch 12 may be rectangular in sha...
579,US11570946,The novel features that are considered charact...
581,US11570948,"Walk-behind lawnmower 70 includes a frame 72, ..."
586,US11570953,"In the following description, reference is mad..."


In [17]:
import time
ts = str(time.time())
patent_data_100.to_csv("clean_data/patents_description_100"+ ts +".csv")

In [17]:
print(patent_data_100['Patent_Number'][586])
print(patent_data_100['Description'][586])

# answer from gpt davinci
# No, the given description does not provide enough information to determine what problem the patent is claiming to solve.

US11570953
In the following description, reference is made to the accompanying drawings which form a part hereof, and which show by way of illustration possible implementations. It is to be understood that other implementations may be utilized, and structural, as well as procedural, changes may be made without departing from the scope of this document. As a matter of convenience, various components will be described using exemplary materials, sizes, shapes, dimensions, and the like. However, this document is not limited to the stated examples and other configurations are possible and within the teachings of the present disclosure. As will become apparent, changes may be made in the function and/or arrangement of any of the elements described in the disclosed exemplary implementations without departing from the spirit and scope of this disclosure. It will be understood that implementations of an enclosed garden bed system are not limited to the specific assemblies, devices and component

In [18]:
print(patent_data_100['Patent_Number'][579])
print(patent_data_100['Description'][579])

# The patent is claiming to solve the problem of under and over dispersal of chemical material across first and second farms, and creating a revised 
# dispersal program to correct the dispersal of material meant for the first farm as needed.

US11570946
The novel features that are considered characteristic of the invention are set forth with particularity in the appended claims. The invention itself; however, both as to its structure and operation together with the additional objects and advantages thereof, are best understood through the following description of the preferred embodiment of the present invention when read in conjunction with the accompanying drawings, wherein: Step 3008 concerns the action taken by the control system in the cloud-based system 32. In step 3008, system 32 determines location and concentration of dispersed material across first and second farms. System 32 then creates a revised dispersal program for the first farm to correct under and over dispersal of chemical material in particular grid locations as needed. System 32 then creates remedial dispersal program for the second farm to correct dispersal of material meant for first farm as needed. Step 3010 concerns the development and uploading of 